In [1]:
import os
if '_cwd_set' not in locals(): locals()['_cwd_set'] = os.chdir(os.path.dirname(os.getcwd()))
import omnifig as fig
fig.initialize()
from src.jimports import *
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, precision_recall_curve
# for all stats at once
from sklearn.metrics import precision_recall_fscore_support
from src.util import set_default_device, repo_root, data_root
from src.interventions import ClassLevelLabelIntervention
from src.dataset import RawCOCO, SimpleCOCO, RawCOCOCaptions, COCOCaptions, COCO, MNIST
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.neighbors import KernelDensity
from scipy.optimize import root_scalar
from scipy.stats import norm
from src.baselines import ConceptAlgebra
from matplotlib import patches as mpatches
from sklearn.metrics import precision_recall_fscore_support

device = set_default_device();

Selecting CUDA device 1 with 48656 MiB free memory and 0% utilization


In [2]:

def roc_threshold(positives, negatives):
    """
    Compute the optimal threshold using the ROC curve.
    
    Args:
        positives: Array of data points from the positive class.
        negatives: Array of data points from the negative class.

    Returns:
        Optimal threshold (float) based on the ROC curve.
    """
    y_true = np.concatenate([np.ones(len(positives)), np.zeros(len(negatives))])
    y_scores = np.concatenate([positives, negatives])
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    optimal_idx = np.argmax(tpr - fpr)
    return thresholds[optimal_idx]

def bayes_threshold(positives, negatives):
    """
    Compute the Bayes threshold using the intersection of Gaussian PDFs.
    Assumes both distributions are Gaussian.

    Args:
        positives: Array of data points from the positive class.
        negatives: Array of data points from the negative class.

    Returns:
        Bayes threshold (float).
    """
    m1, s1 = np.mean(positives), np.std(positives)
    m0, s0 = np.mean(negatives), np.std(negatives)

    def pdf_diff(x):
        return norm.pdf(x, loc=m1, scale=s1) - norm.pdf(x, loc=m0, scale=s0)

    result = root_scalar(pdf_diff, bracket=[m0, m1], method='brentq')
    return result.root


In [3]:
# fullddata = COCO(eval_split=-0.1)
syscfg = fig.create_config('h/ws2')
syscfg.silent=True


In [4]:
loc = '/data/felix/cache/checkpoints/vae128_coco_20250116_222421/ckpt_100000'
loc = '/data/felix/cache/checkpoints/ae128_coco_20250114_230254/ckpt_020000/'
loc = '/data/felix/cache/checkpoints/sae128_coco_20250118_213955/ckpt_100000'
loc = '/ssd/felix/cache/checkpoints/sae512_coco_20250128_121655/ckpt_010000'
loc = Path(loc)

In [5]:
cfg = fig.create_config(*'h/ws2 a/wide norm m/ced d/coco-img'.split(), **{'latent-dim': 512, 'classifier.dropout': 0.1})

In [6]:
trainer = cfg.pull('trainer', silent=True)
model = trainer.model
traindataset = cfg.pull('dataset', silent=True)
valdataset = traindataset.as_eval()
traindataset.prepare(device=device);
valdataset.prepare(device=device)
system = Structured(traindataset, *trainer.gadgetry())
system.mechanize() # sync for gears and spaces
mech = system.mechanics()
model.prepare(device=device);
for p in model.parameters():
    p.requires_grad = False
model.load_checkpoint(path=loc.joinpath('model'));

/home/felix/workspace/clones/omni-learn/omnilearn/machines.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(path, map_location='cpu')


In [7]:
# val_img_ids = valdataset.get_image_id(np.arange(valdataset.size))
# safe_ids = set(fiids.tolist()).intersection(set(val_img_ids.tolist()))
# len(val_img_ids)

In [8]:
# val_img_ids = valdataset.get_image_id(np.arange(valdataset.size))
# fiids = fullddata.get_image_id(np.arange(fullddata.size))
# fiid_map = {fiid: i for i, fiid in enumerate(fiids)}
# # inds = np.array([fiid_map[fiid] for fiid in safe_ids])
# # inds.sort()
# # save_json(inds.tolist(), 'save_inds.json')
# inds = load_json('save_inds.json')
# inds = np.array(inds)
# len(inds), inds[:5]

In [9]:
# valemb = model.encode(valbatch['image_features'])
# lbls = valbatch['label']
# valemb.shape, lbls.shape
# # lbl = 40
# # lbls[:, lbl].sum()
# # pos = emb[lbls[:, lbl]]
# # proj = pos.T @ pos / lbls[:, lbl].sum()
# # proj.shape
# projs = []
# for lbl in range(lbls.shape[1]):
#     pos = valemb[lbls[:, lbl]]
#     proj = pos.T @ pos / lbls[:, lbl].sum()
#     projs.append(proj)
# projs.append(torch.eye(valemb.shape[1], device=valemb.device))
# projs = torch.stack(projs)
# projs.shape

In [10]:
valfulldata = COCO(eval_split=-0.1, dataroot=syscfg.pull('dataroot'))
# fullddata = COCO(eval_split=None, split='val')
valfulldata.prepare(device=device);
fiids = valfulldata.get_image_id(np.arange(valfulldata.size))
valmap = {}
for i, im in enumerate(fiids.tolist()):
    valmap.setdefault(im, []).append(i)
val_img_ids = set(valdataset.get_image_id(np.arange(valdataset.size)).tolist())
valpicks = np.array([options[0] for key, options in valmap.items() if key in val_img_ids])
len(valpicks)

4135

In [11]:
inds = valpicks
valbatch = Context(valfulldata, DictGadget({'index': inds, 'size': len(inds)}))
valbatch

vscore = model.predict(model.encode(valbatch['image_features']))
vy = valbatch['label']

In [12]:
aucs = []
thresholds = []
f1s = []
for ci in tqdm(range(valdataset.label_space.n)):
    points = vscore[:, ci].cpu().numpy()
    gtlabels = vy[:, ci].cpu().numpy()

    threshold = roc_threshold(points[gtlabels], points[~gtlabels])
    thresholds.append(threshold)
    f1 = f1_score(gtlabels, points > threshold)
    f1s.append(f1)
    aucs.append(roc_auc_score(gtlabels, points))
vaucs = torch.as_tensor(aucs).to(device)
vthresholds = torch.as_tensor(thresholds).to(device)
vf1s = torch.as_tensor(f1s).to(device)
vaucs.mean().item(), vthresholds.mean().item(), vf1s.mean().item()

  0%|          | 0/80 [00:00<?, ?it/s]

(0.9802548331114336, -3.3900856971740723, 0.5113652348869506)

In [34]:
modifier = ConceptAlgebra(as_delta=False)
modifier.extract_projections(model.encode(valbatch['image_features']), valbatch['label'])

In [35]:
# test set
fullddata = COCO(eval_split=None, split='val', dataroot=syscfg.pull('dataroot'))
fullddata.prepare(device=device);
fiids = fullddata.get_image_id(np.arange(fullddata.size))
idmap = {}
for i, im in enumerate(fiids.tolist()):
    idmap.setdefault(im, []).append(i)
picks = np.array([options[0] for options in idmap.values()])
fullddata, len(picks)

(COCO[25014](index, text_features, label, image_id, caption_id, caption, image, image_features),
 5000)

In [36]:
inds = picks
batch = Context(fullddata, DictGadget({'index': inds, 'size': len(inds)}))

In [37]:
z = model.encode(batch['image_features'])
y = batch['label']
num_classes = fullddata.label_space.n

score = model.predict(z)
yhat = score > vthresholds.unsqueeze(0)
valid = y.eq(yhat)
valid.shape

torch.Size([5000, 80])

In [38]:
obs_auc = []
obs_pres, obs_recs, obs_f1s, obs_sup = [], [], [], []
for ic in range(num_classes):
    auc = roc_auc_score(y[:, ic].cpu().numpy(), score[:, ic].cpu().numpy())
    obs_auc.append(auc)
    (_,pre), (_,rec), (_,f1), (_,sup) = precision_recall_fscore_support(y[:, ic].cpu().numpy(), score[:, ic].cpu().numpy() > vthresholds[ic].item())
    obs_pres.append(pre.item())
    obs_recs.append(rec.item())
    obs_f1s.append(f1.item())
    obs_sup.append(sup.item())
obs_auc = np.array(obs_auc)
obs_pres = np.array(obs_pres)
obs_recs = np.array(obs_recs)
obs_f1s = np.array(obs_f1s)
obs_sup = np.array(obs_sup)
print(obs_auc.mean(), obs_auc.min(), obs_f1s.mean(), obs_f1s.min(), obs_pres.mean(), obs_recs.mean(), obs_sup.mean())

0.9661585288104874 0.8858969480356834 0.4750030243125334 0.1 0.35273242170661656 0.8783542448585606 182.8875


In [39]:
gti = ClassLevelLabelIntervention()
def compute_interventional_metrics(iy, iscore):
    iyhat = iscore > vthresholds.unsqueeze(0)

    changed_mask = iy != y
    assert changed_mask.any(), f'nothing changed'
    
    correct = iy == iyhat

    sel = changed_mask
    sel = changed_mask & valid
    # (_,c_pre), (_,c_rec), (_,c_f1), (_, c_supp) = precision_recall_fscore_support(torch.ones_like(correct[sel]).cpu().numpy(), correct[sel].cpu().numpy()) # adding and removing are both counted as positive
    (_,c_pre), (_,c_rec), (_,c_f1), (_, c_supp) = precision_recall_fscore_support(iy[sel].cpu().numpy(), iyhat[sel].cpu().numpy())
    c_acc = correct[sel].float().mean().item()

    sel = ~changed_mask
    sel = ~changed_mask & valid
    (_,u_pre), (_,u_rec), (_,u_f1), (_, u_supp) = precision_recall_fscore_support(iy[sel].cpu().numpy(), iyhat[sel].cpu().numpy())
    u_acc = correct[sel].float().mean().item()

    return c_pre, c_rec, c_f1, c_acc, u_pre, u_rec, u_f1, u_acc

In [ ]:
# interventions (training)
torch.manual_seed(11)

num_intv_per_sample = 10

c_accs, u_accs = [], []
c_f1s, c_pres, c_recs, c_supps = [], [], [], []
u_f1s, u_pres, u_recs, u_supps = [], [], [], []

for _ in tqdm(range(num_intv_per_sample)):
    adds, rems = None, None
    adds = gti.sample_add_intervention(y)
    rems = gti.sample_remove_intervention(y)
    
    iz = modifier.apply_intervention(z, adds, rems)
    iscore = model.predict(iz)
    iy = gti.apply_intervention(y, adds, rems)
    
    c_pre, c_rec, c_f1, c_acc, u_pre, u_rec, u_f1, u_acc = compute_interventional_metrics(iy, iscore)
    c_accs.append(c_acc); c_f1s.append(c_f1); c_pres.append(c_pre); c_recs.append(c_rec)
    u_accs.append(u_acc); u_f1s.append(u_f1); u_pres.append(u_pre); u_recs.append(u_rec)

# print(np.array(c_accs).mean(), np.array(u_accs).mean())
# print(np.array(c_f1s).mean(), np.array(c_f1s).std(), np.array(c_pres).mean(), np.array(c_recs).mean())
# print(np.array(u_f1s).mean(), np.array(u_f1s).std(), np.array(u_pres).mean(), np.array(u_recs).mean())
# print(np.array(c_accs).mean(), np.array(c_f1s).mean(), np.array(u_f1s).mean())
itrain_c_acc = np.array(c_accs).mean()
itrain_u_f1 = np.array(u_f1s).mean()


# interventions (not seen during training)
torch.manual_seed(11)

c_accs, u_accs = [], []
c_f1s, c_pres, c_recs, c_supps = [], [], [], []
u_f1s, u_pres, u_recs, u_supps = [], [], [], []

for _ in tqdm(range(num_intv_per_sample)):
    adds, rems = None, None
    adds = gti.sample_add_intervention(y)
    # rems = gti.sample_remove_intervention(y)
    
    iz = modifier.apply_intervention(z, adds, rems)
    iscore = model.predict(iz)
    iy = gti.apply_intervention(y, adds, rems)
    
    c_pre, c_rec, c_f1, c_acc, u_pre, u_rec, u_f1, u_acc = compute_interventional_metrics(iy, iscore)
    c_accs.append(c_acc); c_f1s.append(c_f1); c_pres.append(c_pre); c_recs.append(c_rec)
    u_accs.append(u_acc); u_f1s.append(u_f1); u_pres.append(u_pre); u_recs.append(u_rec)

# print(np.array(c_accs).mean(), np.array(u_accs).mean())
# print(np.array(c_f1s).mean(), np.array(c_f1s).std(), np.array(c_pres).mean(), np.array(c_recs).mean())
# print(np.array(u_f1s).mean(), np.array(u_f1s).std(), np.array(u_pres).mean(), np.array(u_recs).mean())
# print(np.array(c_accs).mean(), np.array(c_f1s).mean(), np.array(u_f1s).mean())
iadd_c_acc = np.array(c_accs).mean()
iadd_u_f1 = np.array(u_f1s).mean()


# interventions (not seen during training)
torch.manual_seed(11)

num_intv_per_sample = 10

c_accs, u_accs = [], []
c_f1s, c_pres, c_recs, c_supps = [], [], [], []
u_f1s, u_pres, u_recs, u_supps = [], [], [], []

for _ in tqdm(range(num_intv_per_sample)):
    adds, rems = None, None
    # adds = gti.sample_add_intervention(y)
    rems = gti.sample_remove_intervention(y)
    
    iz = modifier.apply_intervention(z, adds, rems)
    iscore = model.predict(iz)
    iy = gti.apply_intervention(y, adds, rems)
    
    c_pre, c_rec, c_f1, c_acc, u_pre, u_rec, u_f1, u_acc = compute_interventional_metrics(iy, iscore)
    c_accs.append(c_acc); c_f1s.append(c_f1); c_pres.append(c_pre); c_recs.append(c_rec)
    u_accs.append(u_acc); u_f1s.append(u_f1); u_pres.append(u_pre); u_recs.append(u_rec)

# print(np.array(c_accs).mean(), np.array(u_accs).mean())
# print(np.array(c_f1s).mean(), np.array(c_f1s).std(), np.array(c_pres).mean(), np.array(c_recs).mean())
# print(np.array(u_f1s).mean(), np.array(u_f1s).std(), np.array(u_pres).mean(), np.array(u_recs).mean())
# print(np.array(c_accs).mean(), np.array(c_f1s).mean(), np.array(u_f1s).mean())
irem_c_acc = np.array(c_accs).mean()
irem_u_f1 = np.array(u_f1s).mean()

igen_c_acc = (iadd_c_acc + irem_c_acc) / 2
igen_u_f1 = (iadd_u_f1 + irem_u_f1) / 2



  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

/home/felix/miniconda3/envs/work/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/felix/miniconda3/envs/work/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/felix/miniconda3/envs/work/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/f

  0%|          | 0/10 [00:00<?, ?it/s]

/home/felix/miniconda3/envs/work/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/felix/miniconda3/envs/work/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/felix/miniconda3/envs/work/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/f

0.8681683599948883 0.07379415798093494
0.9962291628122331 0.0696306178257842


/home/felix/miniconda3/envs/work/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/felix/miniconda3/envs/work/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [55]:

print(obs_auc.mean(), obs_auc.min(), obs_f1s.mean(), obs_f1s.min())
print(itrain_c_acc, itrain_u_f1)
print(igen_c_acc, igen_u_f1)

0.9661585288104874 0.8858969480356834 0.4750030243125334 0.1
0.8681683599948883 0.07379415798093494
0.9962291628122331 0.0696306178257842


In [ ]:

# for parity in tqdm(range(num_intv_per_sample)):
#     adds = gti.sample_add_intervention(y)
#     rems = gti.sample_remove_intervention(y)
#     rems[:] = -1
#     # adds[:] = -1
    
#     iz = modifier.apply_intervention(z, adds, rems)
#     iscore = model.predict(iz)
#     iyhat = iscore > vthresholds.unsqueeze(0)

#     iy = gti.apply_intervention(y, adds, rems)
#     changed_mask = iy != y
#     # row_ind = torch.arange(len(iscore), device=device)[adds>0]
#     # row_ind = torch.cat([row_ind, torch.arange(len(iscore), device=device)[rems>0]])
#     # col_ind = torch.cat([adds[adds>0], rems[rems>0]])
#     # changed_mask = torch.zeros_like(iscore)
#     # changed_mask[row_ind, col_ind] = 1
#     # changed_mask = changed_mask.bool()

#     correct = iy == iyhat

#     sel = changed_mask
#     sel = changed_mask & valid
#     (_,c_pre), (_,c_rec), (_,c_f1), (_, c_supp) = precision_recall_fscore_support(torch.ones_like(correct[sel]).cpu().numpy(), correct[sel].cpu().numpy())
#     # (_,c_pre), (_,c_rec), (_,c_f1), (_, c_supp) = precision_recall_fscore_support(iy[sel].cpu().numpy(), iyhat[sel].cpu().numpy())
#     c_pre, c_rec, c_f1, c_supp = c_pre.item(), c_rec.item(), c_f1.item(), c_supp.item()
#     c_acc = (iy[sel] == iyhat[sel]).float().mean().item()
#     c_accs.append(c_acc); c_f1s.append(c_f1); c_pres.append(c_pre); c_recs.append(c_rec); c_supps.append(c_supp)

#     sel = ~changed_mask
#     sel = ~changed_mask & valid
#     (_,u_pre), (_,u_rec), (_,u_f1), (_, u_supp) = precision_recall_fscore_support(iy[sel].cpu().numpy(), iyhat[sel].cpu().numpy())
#     u_pre, u_rec, u_f1, u_supp = u_pre.item(), u_rec.item(), u_f1.item(), u_supp.item()
#     u_acc = (iy[sel] == iyhat[sel]).float().mean().item()
#     u_accs.append(u_acc); u_f1s.append(u_f1); u_pres.append(u_pre); u_recs.append(u_rec); u_supps.append(u_supp)
    
# c_accs = np.array(c_accs); u_accs = np.array(u_accs)
# c_f1s = np.array(c_f1s); c_pres = np.array(c_pres); c_recs = np.array(c_recs); c_supps = np.array(c_supps)
# u_f1s = np.array(u_f1s); u_pres = np.array(u_pres); u_recs = np.array(u_recs); u_supps = np.array(u_supps)
# print(c_accs.mean(), u_accs.mean())
# print(c_f1s.mean(), c_f1s.std(), c_pres.mean(), c_recs.mean(), c_supps.mean())
# print(u_f1s.mean(), u_f1s.std(), u_pres.mean(), u_recs.mean(), u_supps.mean())

In [30]:
iyhat = iscore > vthresholds.unsqueeze(0)

changed_mask = iy != y
assert changed_mask.any(), f'nothing changed'

correct = iy == iyhat

sel = changed_mask
sel = changed_mask & valid
# (_,c_pre), (_,c_rec), (_,c_f1), (_, c_supp) = precision_recall_fscore_support(torch.ones_like(correct[sel]).cpu().numpy(), correct[sel].cpu().numpy()) # adding and removing are both counted as positive
(_,c_pre), (_,c_rec), (_,c_f1), (_, c_supp) = precision_recall_fscore_support(iy[sel].cpu().numpy(), iyhat[sel].cpu().numpy())
c_acc = correct[sel].float().mean().item()

# sel = ~changed_mask
# sel = ~changed_mask & valid
# (_,u_pre), (_,u_rec), (_,u_f1), (_, u_supp) = precision_recall_fscore_support(iy[sel].cpu().numpy(), iyhat[sel].cpu().numpy())
# u_acc = correct[sel].float().mean().item()


In [29]:
correct[sel]

tensor([True, True, True,  ..., True, True, True], device='cuda:1')

In [ ]:
# yhat valid
iyhat = iscore > vthresholds.unsqueeze(0)
correct = iy.eq(iyhat)


In [181]:
# accuracy averaged across classes in observational distribution
valid.mul(y).float().sum(0).div(y.float().sum(0)).mean().item()

0.8786563873291016

In [183]:
# accuracy of changed, accuracy of unchanged
correct[changed_mask & valid].float().mean(), correct[~changed_mask & valid].float().mean()

(tensor(0.8805, device='cuda:1'), tensor(0.9259, device='cuda:1'))

In [184]:
precision_recall_fscore_support(iy[changed_mask & valid].cpu().numpy(), iyhat[changed_mask & valid].cpu().numpy())

(array([0.78748628, 0.99591392]),
 array([0.99583565, 0.79065744]),
 array([0.87949001, 0.88149488]),
 array([3602, 4624]))

In [185]:
precision_recall_fscore_support(iy[~changed_mask & valid].cpu().numpy(), iyhat[~changed_mask & valid].cpu().numpy())

(array([0.97873884, 0.05217978]),
 array([0.94472459, 0.12913235]),
 array([0.96143097, 0.07432595]),
 array([355565,   8379]))

In [13]:
# ci = 7
# ci = random.randint(0, len(valdataset.label_space.class_names)-1)
# # ci = 0
# # print(valdataset.label_space.class_names[ci])

# # plot the roc curve
# positives = vscore[vy[:, ci], ci].cpu().numpy()
# negatives = vscore[~vy[:, ci], ci].cpu().numpy()

# fpr, tpr, thresholds = roc_curve(np.concatenate([np.ones(len(positives)), np.zeros(len(negatives))]),
#                                     np.concatenate([positives, negatives]))
# plt.plot(fpr, tpr, label='ROC curve')
# plt.title(valdataset.label_space.class_names[ci])

# points = vscore
# gtlabels = vl

# p_samples = points[gtlabels[:, ci], ci].cpu().numpy()
# n_samples = points[~gtlabels[:, ci], ci].cpu().numpy()
# samples = np.concatenate([p_samples, n_samples])
# gt = np.concatenate([np.ones(len(p_samples)), np.zeros(len(n_samples))])


# plt.hist(p_samples, bins=100, alpha=0.5, label='original');
# plt.hist(n_samples, bins=100, alpha=0.5, label='original');

# f1s = []
# leg = []
# for i, threshold_fn in enumerate([roc_threshold, bayes_threshold]):
#     threshold = threshold_fn(p_samples, n_samples)
#     # print(f"{threshold_fn.__name__} threshold: {threshold:.3f}")
#     f1 = f1_score(gt, samples > threshold)
#     plt.axvline(threshold, label=f"{threshold_fn.__name__} threshold", c=f'C{i}', linestyle='--')
#     # add the f1 to the legend
#     f1s.append(f1)
#     leg.append(mpatches.Patch(color=f'C{i}', label=f"{threshold_fn.__name__} threshold (F1={f1:.3f})"))
# # sorted by f1
# plt.title(valdataset.label_space.class_names[ci])
# plt.legend(handles=[leg[i] for i in np.argsort(f1s)[::-1]]);
# plt.tight_layout();


In [ ]:
# def other_inds(N, r1, r2=None):
#     full = torch.arange(1,N,device=r1.device).unsqueeze(0).expand(r1.shape[0], -1).clone()
    
#     m1 = r1.view(-1, 1)
#     if r2 is None:
#         return full.sub(r1.view(-1, 1)).remainder(N)
    
#     m2 = r2.view(-1, 1)
#     return full.add(m2-m1-1).remainder(N-1)[:,:-1].add(m1+1).remainder(N)
# c_inds = torch.stack([adds, rems], dim=-1)
# u_inds = other_inds(80, adds, rems)
# c_inds.shape, u_inds.shape


In [19]:
# select the elements corresponding to c_inds in iscore
c_score = iscore.cpu().gather(1, c_inds.cpu())
c_score.shape

RuntimeError: index -1 is out of bounds for dimension 1 with size 80

In [357]:
N = 80
r1 = torch.tensor([0])
r2 = torch.tensor([1])
full = torch.arange(1,N).unsqueeze(0).expand(r1.shape[0], -1).clone()
m1 = r1.view(-1, 1)
m2 = r2.view(-1, 1)
full

tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
         19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
         37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
         55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
         73, 74, 75, 76, 77, 78, 79]])

In [351]:
for _ in range(1000):
    # modulo 80
    i, j = adds[0].item(), removes[0].item()
    # print(i,j)
    i = random.randint(0, 79)
    j = random.randint(0, 78) + i
    j = j % 80
    # i = 3
    # j = 0
    i, j
    i, j = (i, j) if i < j else (j, i)
    s1, s2 = j-i-1, i+1
    sel = torch.arange(1,80).add(s1).remainder(79)[:-1].add(s2).remainder(80)
    assert j not in sel.tolist() and i not in sel.tolist()
# sel[:-1].add_(2).remainder_(80)
# print(len(sel))
# 71, 3 -> 11, 72
# 53, 51 -> 77, 54
# 0, 1 -> 0, 1
# 0, 10 -> 9, 1
# 0, 79 -> 78, 1
# 1, 79 -> 77, 2
# 1, 0 -> -1, 2
# 1, 0 -> 78, 2
# 2, 0 -> -2, 3
print(f'{i}, {j} -> {s1}, {s2} {i not in sel.tolist() and j not in sel.tolist()}')
print({x for x in range(80)} - set(sel.tolist()))
sel

9, 19 -> 9, 10 True
{9, 19}


tensor([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
        38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
        56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73,
        74, 75, 76, 77, 78, 79,  0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12,
        13, 14, 15, 16, 17, 18])

In [ ]:
i = random.randint(0, len(iz)-1)

delta = iscore.sub(score)
addi = adds[i].item()
remi = removes[i].item()
# print(addi, remi)
print(tabulate([('original', score[i, addi].cpu().int().numpy(), score[i,remi].cpu().int().numpy()),
('delta', delta[i,addi].int().cpu().numpy(), delta[i,remi].int().cpu().numpy()),
('end', iscore[i, addi].cpu().int().numpy(), iscore[i,remi].cpu().int().numpy())], headers=['', 'add', 'remove'], tablefmt='fancy_grid'))

# print(delta[i].int().cpu().numpy())
# print(y[i].cpu().numpy())
# print(score[i].cpu().int().numpy())
# print(score[i].ge(0).cpu().numpy())
# print(iscore[i].cpu().int().numpy())
# print(iscore[i].ge(0).cpu().numpy())


╒══════════╤═══════╤══════════╕
│          │   add │   remove │
╞══════════╪═══════╪══════════╡
│ original │    -9 │        5 │
├──────────┼───────┼──────────┤
│ delta    │    -6 │      -40 │
├──────────┼───────┼──────────┤
│ end      │   -16 │      -35 │
╘══════════╧═══════╧══════════╛


In [51]:
y[i]

tensor([ True, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False],
       device='cuda:1')

In [43]:
adds[0].item(), removes[0].item()

(71, 3)

In [41]:
valdataset.label_space.class_names

('person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'couch',
 'potted plant',
 'bed',
 'dining table',
 'toilet',
 'tv',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush')

In [16]:


ic = 2
y[:,ic].float().mean()

dz = z @ projs[ic]
dz = z + (-1) ** y[:,ic].unsqueeze(-1) * dz
dz = dz / dz.norm(dim=-1, keepdim=True)
dz.shape 

torch.Size([5000, 80])

In [45]:
obs_results = []
intv_results = []

obs_score = model.predict(z).cpu().numpy()

for ic in tqdm(range(y.shape[1])):
    dz = z @ projs[ic]
    dz = z + (-1) ** y[:,ic].unsqueeze(-1) * dz
    dz = dz / dz.norm(dim=-1, keepdim=True)
    intv_score = model.predict(dz).cpu().numpy()
    obs_results.append(roc_auc_score(gt, obs_score[:, i]))
    intv_row = []
    for i in range(y.shape[1]):
        gt = y[:, i].cpu().numpy()
        if i == ic:
            gt = ~gt
        intv_row.append(roc_auc_score(gt, intv_score[:, i]))
    intv_results.append(intv_row)
obs_results = np.array(obs_results)
intv_results = np.array(intv_results)
obs_results.shape, intv_results.shape

((80,), (80, 80))

In [14]:
model.predict(emb)[idx]

tensor([  9.1324,  -9.0957, -11.5980, -13.6753, -15.8401, -17.5224, -11.4130,
        -10.6889,  -9.6972, -12.5531, -19.4821, -16.7362, -18.4169,  -8.7252,
        -11.3438,  -8.1346,  -8.7017,  -8.9816, -15.1360, -16.9146, -14.9189,
        -13.8886, -17.9677, -13.7450,  -7.1458, -11.8310,  -5.7632,  -7.5476,
        -10.2915, -14.5164, -22.3432, -23.1013, -12.3706, -22.9805, -13.9289,
        -18.5452, -22.8026, -20.0341, -11.0255,  -0.3999,  -6.2802,  -0.1928,
         -8.1223,  -6.1516,  -6.1214,  -2.8473, -10.8983,  -6.5904, -11.2488,
         -7.8607, -17.4817, -12.2376, -13.9882, -16.8319, -15.4528, -13.8059,
         -5.0010,  -7.1793,   0.6025,  -6.3523,  -8.7216,  -0.9808,  -8.0420,
         -6.2589, -13.6159,  -7.4622, -11.5423,  -1.2052,  -9.2549,  -9.9871,
        -12.3797,   5.3217, -10.4644,  -2.8025,  -4.4475,  -0.2001,  -5.8738,
         -7.3922,  -3.4281,  -1.3222], device='cuda:0')

In [15]:
new = (projs[-4] - projs[-5]) @ emb[idx]
new = new / new.norm()
new.shape

torch.Size([128])

In [16]:
model.predict(new.unsqueeze(0))

tensor([[  3.9024, -10.6718,  -6.7774, -13.5819,  -5.4185,  -5.7317, -10.2014,
          -8.8080, -14.0354,  -9.1711, -16.1837,  -8.8150, -13.6464, -10.6974,
          -6.5013, -16.7903, -10.8953, -11.2102, -15.7333,  -9.0341, -13.8807,
         -17.1458, -19.8320, -17.6132,  -6.9916, -12.8268,  -8.4250, -12.4215,
         -12.9349,  -9.5340, -12.5550, -14.9554, -10.9486,  -7.8032, -15.0483,
         -12.4928, -14.6519, -15.7950, -19.9726,  -8.9246, -21.3170, -14.5983,
          -9.4339,  -6.3230, -10.8002,  -8.1817, -10.1597, -11.4045,  -9.9101,
         -14.0311, -13.2979,  -7.0450, -10.4698, -13.5920, -16.6083,  -6.0683,
         -13.4473, -19.2462, -16.3298, -23.2252, -12.8410, -21.5716, -14.6681,
         -14.6640, -12.4886, -14.2852, -11.2389,  -8.4424, -14.2416, -15.9075,
          -8.9048, -10.5014, -15.6897, -11.2378, -14.1076, -27.9013,   0.8805,
         -14.6172, -19.6178, -15.6648]], device='cuda:0')

In [17]:
model.predict(emb).shape

torch.Size([4135, 80])

In [19]:
pred = model.predict(emb)
i = 0
thrs = []
aucs = []
for i in range(pred.shape[1]):
    fpr, tpr, thresholds = roc_curve(lbls[:,i].cpu().numpy(), pred[:,i].cpu().numpy())
    auc = roc_auc_score(lbls[:,i].cpu().numpy(), pred[:,i].cpu().numpy())
    thr = thresholds[(tpr-fpr).argmax()]
    thrs.append(thr)
    aucs.append(auc)
thrs = torch.as_tensor(thrs).to(device)
vaucs = torch.as_tensor(aucs).to(device)
vaucs.mean(), thrs.mean()

(tensor(0.9789, device='cuda:0', dtype=torch.float64),
 tensor(-3.2697, device='cuda:0'))

In [58]:
z = model.encode(batch['image_features'])
y = batch['label']
score = model.predict(z)
yp = score.sub(thrs.unsqueeze(0)).ge(0)
score.shape

torch.Size([5000, 80])

In [54]:

aucs = []
for i in range(pred.shape[1]):
    auc = roc_auc_score(y[:,i].cpu().numpy(), score[:,i].cpu().numpy())
    aucs.append(auc)
aucs = torch.as_tensor(aucs).to(device)
aucs.mean()

tensor(0.9674, device='cuda:2', dtype=torch.float64)

In [59]:
f1_score(y[:,idx].cpu().numpy(), yp[:,idx].cpu().numpy())

np.float64(0.2764505119453925)

In [65]:
outs = []
for idx in range(y.shape[1]):
    p, r, f, _ = precision_recall_fscore_support(y[:,idx].cpu().numpy(), yp[:,idx].cpu().numpy())
    outs.append((p[1], r[1], f[1]))
outs = torch.as_tensor(outs).to(device)
outs.shape

torch.Size([80, 3])

In [68]:
outs[:, -1]#.mean()

tensor([0.9073, 0.2378, 0.5214, 0.5544, 0.8257, 0.5926, 0.6576, 0.4215, 0.6176,
        0.3743, 0.2765, 0.1743, 0.1308, 0.2748, 0.2030, 0.7054, 0.3593, 0.5011,
        0.6087, 0.4420, 0.9438, 0.7500, 0.9390, 0.8919, 0.3023, 0.3384, 0.3568,
        0.4231, 0.2056, 0.4862, 0.8679, 0.4653, 0.5704, 0.7330, 0.6690, 0.8597,
        0.6244, 0.8581, 0.9174, 0.4437, 0.3298, 0.4681, 0.3543, 0.4094, 0.3725,
        0.4373, 0.4373, 0.2147, 0.4462, 0.2431, 0.4769, 0.4409, 0.2479, 0.7949,
        0.3661, 0.4580, 0.5067, 0.3570, 0.1854, 0.4668, 0.5943, 0.6712, 0.5326,
        0.5668, 0.6884, 0.5345, 0.4138, 0.3149, 0.1889, 0.4283, 0.2286, 0.6654,
        0.4093, 0.3803, 0.2857, 0.3659, 0.1366, 0.4501, 0.0613, 0.1299],
       device='cuda:2', dtype=torch.float64)

In [63]:
outs[0]

(array([0.86018117, 0.95002032]),
 array([0.94668401, 0.86817675]),
 array([0.90136195, 0.9072565 ]),
 array([2307, 2693]))